In [1]:
import ubermag
print(ubermag.__version__)
import discretisedfield as df
import plotly
print(plotly.__version__)
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
#import matplotlib.gridspec as gridspec
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng

0.62.0
5.9.0


In [2]:
system = "BP" #"BV", "CV"
for system in ["BP", "BV", "CV"]:
    model ="MM" # "HB"
    sample = "cylindrical" #"rectangular"
    nt = 20
    file = system+"/"+model+"_"+sample+"/"+system+"_relaxed_nt_"+str(nt)+".omf"
    
    field_read = df.Field.fromfile(file)
    mag = field_read.array
    
    a=0.4679
    
    x_ = np.linspace(0., 200*a, mag.shape[0])
    y_ = np.linspace(0., 200*a, mag.shape[1])
    z_ = np.linspace(0., 50*a, mag.shape[2])
    x, y, z = np.meshgrid(x_, y_, z_, indexing='xy')   
    
    magx = mag[:,:,:,0]
    magy = mag[:,:,:,1]
    magz = mag[:,:,:,2]
    magzabs = np.absolute(magz)
    xrav = np.ravel(x)
    yrav=np.ravel(y)
    zrav = np.ravel(z)
    magxrav = np.ravel(magx)
    magyrav = np.ravel(magy)
    magzrav = np.ravel(magz)
    magzravabs = np.absolute(magzrav)
    deleteindicesUP = np.argwhere(magzrav <0.9) #up-magnetized
    xravUP = np.delete(xrav,deleteindicesUP)
    yravUP = np.delete(yrav,deleteindicesUP)
    zravUP = np.delete(zrav,deleteindicesUP)
    magxravUP = np.delete(magxrav,deleteindicesUP)
    magyravUP = np.delete(magyrav,deleteindicesUP)
    magzravUP = np.delete(magzrav,deleteindicesUP)  
    magzravabsUP = np.delete(magzravabs, deleteindicesUP)
    deleteindicesDOWN = np.argwhere(magzrav >-0.9) #down-magnetzied
    xravDOWN = np.delete(xrav,deleteindicesDOWN)
    yravDOWN = np.delete(yrav,deleteindicesDOWN)
    zravDOWN = np.delete(zrav,deleteindicesDOWN)
    magxravDOWN = np.delete(magxrav,deleteindicesDOWN)
    magyravDOWN = np.delete(magyrav,deleteindicesDOWN)
    magzravDOWN = np.delete(magzrav,deleteindicesDOWN)  
    magzravabsDOWN = np.delete(magzravabs, deleteindicesDOWN) 
    deleteindicesIP = np.argwhere(magzravabs > 0.8) #IP magnetization
    xravIP = np.delete(xrav,deleteindicesIP)
    yravIP = np.delete(yrav,deleteindicesIP)
    zravIP = np.delete(zrav,deleteindicesIP)
    magxravIP = np.delete(magxrav,deleteindicesIP)
    magyravIP = np.delete(magyrav,deleteindicesIP)
    magzravIP = np.delete(magzrav,deleteindicesIP)  
    magzravabsIP = np.delete(magzravabs, deleteindicesIP) 
    rng = default_rng()
    numbers = rng.choice(xravIP.shape[0], size=200, replace=False)
    
    
    xravIPplot = np.take(xravIP, numbers)
    yravIPplot = np.take(yravIP, numbers)
    zravIPplot = np.take(zravIP, numbers)
    
    
    magxravIPplot = np.take(magxravIP, numbers)
    magyravIPplot = np.take(magyravIP, numbers)
    magzravIPplot = np.take(magzravIP, numbers)
    
    
    ###plot poster Fig 4a)
    trace1 = go.Isosurface(
        x=xrav,
        y=yrav,
        z=zrav,
        value=magzrav,
        isomin= -0.85,
        isomax= -0.8,
        #surface_count=1, # number of isosurfaces, 2 by default: only min and max
        #surface_fill=0.8,
        #cauto = False,
        cmin = -1,
        cmax=1,
        colorscale='RdBu',
        opacity=0.5,
        colorbar=None
            )
    trace2 = go.Isosurface(
        x=xrav,
        y=yrav,
        z=zrav,
        value=magzrav,
        isomin= 0.8,
        isomax= 0.85,
        #surface_count=1,
        cmin = -1,
        cmax=1,
        colorscale='RdBu',
        opacity=0.5,
        colorbar=None
            )
    
    trace3 =  go.Cone(
        x=xravIPplot,
        y=yravIPplot,
        z=zravIPplot,
        u=magyravIPplot,
        v=magxravIPplot,
        w=magzravIPplot,
        cauto = False,
        colorscale=[[0, "rgb(255,255,255)"],[1,"Grey"]],
        #sizemode="absolute",
            lighting_ambient=0.1, name="lighting.ambient:0.3",#0.3
        opacity=0.1,#0.2
        sizeref=0.775#0.75,
    
        )
    
    trace4 = go.Isosurface(
        x=xrav,
        y=yrav,
        z=zrav,
        value=magzrav,
        isomin= -0.9,
        isomax= 0.9,
        surface_count=1,
        cmin = -1,
        cmax = 1,
        colorscale='RdBu',
        opacity=0.2,
        colorbar=None
            )
    fig = make_subplots()
    fig.update_traces(showlegend=False)
    fig.update_layout(showlegend=False)
    fig.update(layout_showlegend=False)
    #fig.update_traces(marker_showscale=False)
    fig.update(layout_coloraxis_showscale=False)
    
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace3)
    fig.add_trace(trace4)
    
    
    fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=0.25),
                        camera_eye=dict(x=1.2, y=0.8, z=0.6),
                            xaxis_title="x (nm)",   yaxis_title= "y (nm)", zaxis_title="z (nm)",
                                zaxis= dict(tickvals=[0.0, 12.5, 25.0] )), 
                              xaxis = dict(tickvals=[0, 20, 40, 60, 80]),
                          yaxis = dict(tickvals=[0, 20, 40, 60, 80])
                     )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.update_traces(showscale=False)
    fig.update_coloraxes(showscale=False)
    fig.write_image("../Figures/"+model+"_"+system+"_"+sample+"_"+str(nt)+".svg")
    fig.write_image("../Figures/"+model+"_"+system+"_"+sample+"_"+str(nt)+".pdf")
    fig.write_image("../Figures/"+model+"_"+system+"_"+sample+"_"+str(nt)+".png")
    fig.show()
    #fig.close()

FileNotFoundError: [Errno 2] No such file or directory: 'BP/HB_cylindrical/BP_relaxed_nt_20.omf'